In [1]:
##Уравнение Пуассона в прямоугольнике
##0) Ряд Фурье для решения по собственным функциям

In [150]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

a1=0
b1=1
a2=0
b2=1

l1=b1-a1
l2=b2-a2

N1=20
N2=20

n_points=N1
mesh_x=np.linspace(0, 1, N1)
mesh_y=np.linspace(0, 1, N2)

h1=l1/N1
h2=l2/N2

def f(x, y):
    return np.sin(x*y)

def forward_coefs(u=1):
    m=np.zeros((N1, N2))
    coefs=np.zeros((N1, N2))
    
    for k2 in range(1, N2):
        for i in range(1, N1):
            val=0
            for j in range(1, N2):
                val+=f(i*h1, j*h2)*np.sin(k2*j*np.pi/N2)
            m[k2, i]=val
    
    
    
    for k1 in range(1, N1):
        for k2 in range(1, N2):
            val=0
            for i in range(1, N1):
                val+=m[k2, i]*np.sin(k1*i*np.pi/N1)
            coefs[k1, k2]=val
        
    
    return coefs


def backward_coefs(u=1):
    m=forward_coefs(u=1)
    b=np.zeros((N1, N2))
    values=np.zeros((N1, N2))
    
    
    
    for k2 in range(1, N2):
        for i in range(1, N1):
            val=0
            for k1 in range(1, N1):
                s1=(np.sin(k1*h1*np.pi/(2*l1)))**2
                s2=(np.sin(k2*h2*np.pi/(2*l2)))**2
                s=np.sin(k1*np.pi*i/N1)
                c1=4/h1**2
                c2=4/h2**2
                val+=m[k1, k2]*s/(c1*s1+c2*s2)
            b[k2, i]=val
    
    
    
    for i in range(1, N1):
        for j in range(1, N2):
            val=0
            for k2 in range(1, N2):
                val+=b[k2, i]*np.sin(k2*j*np.pi/N2)
            val=val*4/(N1*N2)
            values[i, j]=val
        
    
    return values


In [2]:
##Объявлем коэффициенты оператора левой части

In [127]:
def k11(x,y):
    return 1

In [151]:
def k22(x, y):
    return 1+y**2

In [152]:
def K11(x,y):
    return 1
def K22(x,y):
    return 1+y**2

In [155]:

def build_mesh(u=1):
    x = np.linspace(a1, b1, N1)
    y = np.linspace(a2, b2, N2)
    mesh=[]
    for t in x:
        for p in y:
            mesh.append([t, p])
        
    return np.array(mesh)

In [3]:
##Правая часть

In [156]:
def F(u=1):
    mesh=build_mesh(u=1)

    def f_r(x):
        return 2*(x[0]-1)*(x[0])*(3*x[1]**2-x[1]+1)+x[1]*(x[1]-1)
    
    F=[]
    
    for x in mesh:
        if x[0]==a1 or x[0]==b1 or x[1]==a2 or x[1]==b2:
            F.append(f_r(x))
        
        else:
            F.append(f_r(x))
            
    return np.array(F)

In [4]:
##Аналитическое решение

def U(u=1):
    mesh=build_mesh(u=1)
    
    
    def solution(x):
        return x[0]*(x[0]-1)*x[1]*(x[1]-1)
    
    
    U=[]
    
    for x in mesh:
        if x[0]==a1 or x[0]==b1 or x[1]==a2 or x[1]==b2:
            U.append(solution(x))
        
        else:
            U.append(solution(x))
        
    return np.array(U)

In [5]:
#Матрица преобуславлевателя 

In [158]:
def matrix_B(n_points, mesh_x, mesh_y):
    matrix_n = n_points**2
    h = abs(mesh_x[1] - mesh_x[0])
    h=1/N1
    
    diag_1 = np.diag(np.array([4]*matrix_n), 0)
    diag_2 = np.diag(np.array([-1]*(matrix_n - 1)), 1)
    diag_3 = np.diag(np.array([-1]*(matrix_n - 1)), -1)
    diag_4 = np.diag(np.array([-1]*(matrix_n - n_points)), n_points)
    diag_5 = np.diag(np.array([-1]*(matrix_n - n_points)), -n_points)
    
    i = n_points
    while (i < matrix_n - 1):
        diag_2[i-1] = 0
        diag_3[i] = 0
        i += n_points
    
    return (diag_1 + diag_2 + diag_3 + diag_4 + diag_5) / pow(h,2)

In [6]:
##Матрицы разностных операторов
def Lambda_yy(n_points, mesh_x, mesh_y):
    matrix_n = n_points**2
    hx = mesh_x[1] - mesh_x[0]
    hx=h1
    
    Lyy = np.zeros((matrix_n, matrix_n))
    
    for i in range(matrix_n):
        Lyy[i][i] = (2*K22(mesh_x[i%n_points], mesh_y[i//n_points]) + 
                     K22(mesh_x[i%n_points] + hx, mesh_y[i//n_points]) + 
                     K22(mesh_x[i%n_points] - hx, mesh_y[i//n_points])) / 2
        
    for i in  range(1, matrix_n):
        Lyy[i][i-1] = (K22(mesh_x[i%n_points], mesh_y[i//n_points]) + K22(mesh_x[i%n_points] + hx, mesh_y[i//n_points])) / 2
        
    for i in range(1, matrix_n):
        Lyy[i-1][i] = (K22(mesh_x[i%n_points], mesh_y[i//n_points]) + K22(mesh_x[i%n_points] - hx, mesh_y[i//n_points])) / 2
        
    return Lxx / hx**2

def Lambda_yy_2(n_points, mesh_x, mesh_y):
    matrix_n = n_points**2
    h = abs(mesh_x[1] - mesh_x[0])
    h=h1
    
    K_values_main = np.zeros(matrix_n)
    for row in range(n_points):
        for col in range(n_points):
            i = row * n_points + col
            K_values_main[i] = -(2 * K22(mesh_x[col], mesh_y[row]) +
                                K22(mesh_x[col] + h, mesh_y[row]) +
                                K22(mesh_x[col] - h, mesh_y[row])) / 2
    
    K_values_down = np.zeros(matrix_n - 1)
    for ind in range(1, matrix_n):
        row = ind // n_points
        col = ind % n_points
        K_values_down[ind-1] = (K22(mesh_x[col], mesh_y[row]) + 
                         K22(mesh_x[col] - h, mesh_y[row])) / 2
        
    K_values_up = np.zeros(matrix_n - 1)
    for ind in range(matrix_n - 1):
        row = ind // n_points
        col = ind % n_points
        K_values_up[ind] = (K22(mesh_x[col], mesh_y[row]) + 
                       K22(mesh_x[col] + h, mesh_y[row])) / 2
        
    main_diag = np.diag(K_values_main)
    up_diag = np.diag(K_values_up, 1)
    down_diag = np.diag(K_values_down, -1)
    
    i = n_points
    while (i < matrix_n - 1):
        up_diag[i-1] = 0
        down_diag[i] = 0
        i += n_points
    
    return (main_diag + up_diag + down_diag) / h1**2
    

def Lambda_xx(n_points, mesh_x, mesh_y):
    matrix_n = n_points**2
    h = mesh_x[1] - mesh_x[0]
    h=h1
    
    diag_1 = np.diag(np.array([-2]*matrix_n), 0)
    diag_2 = np.diag(np.array([1]*(matrix_n - n_points)), n_points)
    diag_3 = np.diag(np.array([1]*(matrix_n - n_points)), -n_points)
        
    return (diag_1 + diag_2 + diag_3) / h**2

def matrix_A(n_points, mesh_x, mesh_y):
    h = abs(mesh_x[1] - mesh_x[0])
    
    Lxx = Lambda_xx(n_points, mesh_x, mesh_y)
    Lyy = Lambda_yy_2(n_points, mesh_x, mesh_y)
    
    return (- Lxx - Lyy)

In [7]:
##Скалярное произведение сеточных функций

In [160]:
def dot_prod(v1, v2):
    product=0
    for i in range(1, N2):
        for j in range(1, N1):
            product+=v1[i,j]*v2[i, j]*h1*h2
    return product

In [10]:
##Итерационный метод с адаптивной скоростью

In [161]:
def iter_step(A, B, v_n, alpha_n1, b):
    B=np.linalg.inv(B)
    r_n = np.matmul(A, v_n) - b
    v_n1=v_n - alpha_n1*np.matmul(B, r_n)
    return v_n1

In [12]:
#Вычисляем скорость метода

In [162]:
def calc_alpha(A, B, r_n):
    B=np.linalg.inv(B)
    p=np.matmul(B,r_n)
    
    numerator=np.dot(r_n, p)
    denumerator=np.dot(np.matmul(A, p), p)
    
    alpha_n1=numerator/denumerator
    
    return alpha_n1

In [163]:
def d(A, y,f):
    
    prod = np.matmul(A, y)
    d = np.max(np.abs(prod- f))
    
    return d

In [13]:
def find_solution(n_iter):
    j=1
    mesh=build_mesh(u=1)
    B=laplas_B(n=N1, h=h1)
    A=matrix_A(n_points, mesh_x, mesh_y)
    u_prev=np.zeros((N1**2,))
    u_new=u_prev
    f=F(u=1)
    S=U(u=1)
    alpha=1/3
    eps=1e-5
    counter=0
    r_n = np.matmul(A, u_new) - f
    while np.max(np.abs(r_n))>eps:
        u_prev=u_new
        u_new = iter_step(A, B, u_prev, alpha, f)
        
        
        r_n = np.matmul(A, u_new) - f
        alpha = calc_alpha(A, B, r_n)
        j+=1
        print(np.max(np.abs(r_n)))
        counter+=1
    return counter
        
        

In [14]:
##Запускаем итерационный метод с выводом нормы невязки

In [166]:
find_solution(1)

0.8973481172555589
0.17540243142092082
0.05888686536543619
0.013525493158821233
0.0047190296281863775
0.0011138962071615133
0.0003837554818919564
8.517986800216093e-05
2.8267406118498428e-05
6.1353444347211905e-06


10